In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow        as tf
import numpy             as np

import pathlib
import config
import random
import data

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model

"""
Documentation:
- matplotlib
    - pyplot
        1. bar(), grid(), show(), xticks(), ylim()
            - https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.html?highlight=pyplot#module-matplotlib.pyplot
- numpy
    1. argsort()
        - https://numpy.org/doc/stable/reference/generated/numpy.argsort.html?highlight=argsort#numpy.argsort
    2. expand_dims()
        - https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html?highlight=expand_dims#numpy.expand_dims
    3. load()
        - https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load
    4. pad()
        - https://numpy.org/doc/stable/reference/generated/numpy.pad.html?highlight=pad#numpy.pad
- pathlib
    1. Path(), /, glob(), .name, .stem
        - https://docs.python.org/3/library/pathlib.html
- random
    1. randint()
        - https://docs.python.org/3/library/random.html#random.randint
- tensorflow
    - data.Dataset
        1. batch(), from_tensor_slices(), list_files(), map(), prefetch()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/data/Dataset
    - keras
        - applications.inception_v3
            1. preprocess_input()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/applications/inception_v3
        - models
            1. load_model()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/models/load_model
            2. Sequential()
                1. predict()
                    - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/Sequential
        - preprocessing.image
            1. img_to_array(), load_img()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/preprocessing/image

Sources:
    1. Making predictions and displaying them using matplotlib bar plots.
        - https://www.tensorflow.org/tutorials/keras/classification#make_predictions
        - https://www.tensorflow.org/tutorials/keras/classification#verify_predictions
"""

In [ ]:
dataObj    = data.Data()
classNames = dataObj.classes
numClasses = dataObj.numClasses

In [ ]:
"""
Function Name: getBestSavedModel
Number of parameters: 1
List of parameters:
    1. modelCheckpointPath | pathlib.Path | Path to the CNN model checkpoints.
Pre-condition:
    1. modelCheckpointPath exists.
Post-condition:
    1. Returns the path to the best model in the 'modelCheckpointPath' directory. The best model, is the one
       that has the smallest loss.
"""
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = sorted(modelCheckpointPath.glob("*"))
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
"""
Function Name: predictCNN
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Makes predictions using a CNN model.
    2. Outputs a bar chart of topk predictions. 
    3. Nothing is returned.
"""
def predictCNN():
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    def plotPredictions(predictions, predictedLabels):
        plt.grid(False)
        plot = plt.bar(range(len(predictions)), predictions, color="#777777")
        plt.xticks(range(len(predictedLabels)), predictedLabels, rotation=20)
        plt.ylim([0, 1])
        plot[0].set_color('blue')
        plt.show()
    
    
    dataSetType   = "Test"
    cf            = config.Config()
    rootPath      = pathlib.Path(cf.rootPath)
    framesPath    = pathlib.Path(cf.framesPath)
    dataDirectory = framesPath/dataSetType
    framesPaths   = list(sorted(dataDirectory.glob('*/*.jpg')))
    frameCount    = len(framesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks'/'CNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath      = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    topk = 3
    numFrames = 5
    for i in range(numFrames):
        randomIndex     = random.randint(0, frameCount - 1)
        randomFramePath = framesPaths[randomIndex]
        
        actualClass     = randomFramePath.parts[-2]
        predictions     = getPredictions(savedModel, randomFramePath)
        topkindecies    = predictions.argsort()[topk*-1:][::-1]
        topkpredictions = predictions[topkindecies]
        
        predictedLabels    = []
        for index in topkindecies:
            predictedLabels.append(classNames[index])
        
        print("Video name: ", "_".join(randomFramePath.stem.split("_")[:-1]))
        print("Actual class: ", actualClass)
        print(f'Predicted class: {classNames[topkindecies[0]]} - {predictions[topkindecies[0]] *100}%')
        plotPredictions(topkpredictions, predictedLabels)
        print()
        
predictCNN()

In [ ]:
"""
Function Name: predictRNN
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Makes predictions using a RNN model.
    2. Outputs a bar chart of topk predictions. 
    3. Nothing is returned.
"""
def predictRNN():
    def getPredictions(model, sequencePath):
        maxFrameCount = dataObj.getMaxFrameCount()
        sequence      = np.load(sequencePath)
        sequence      = np.pad(sequence, ((0, maxFrameCount - len(sequence)), (0, 0)), 'edge')
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    def plotPredictions(predictions, predictedLabels):
        plt.grid(False)
        plot = plt.bar(range(len(predictions)), predictions, color="#777777")
        plt.xticks(range(len(predictedLabels)), predictedLabels, rotation=20)
        plt.ylim([0, 1])
        plot[0].set_color('blue')
        plt.show()
    
    dataSetType    = "Test"
    cf             = config.Config()
    rootPath       = pathlib.Path(cf.rootPath)
    dataDirectory  = rootPath/'Sequences'/dataSetType
    sequencesPaths = list(sorted(dataDirectory.glob('*/*.npy')))
    sequenceCount  = len(sequencesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks'/'RNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath      = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    topk = 3
    numSequences = 5
    for i in range(numSequences):
        randomIndex        = random.randint(0, sequenceCount - 1)
        randomSequencePath = sequencesPaths[randomIndex]
        
        actualClass        = randomSequencePath.parts[-2]
        predictions        = getPredictions(savedModel, randomSequencePath)
        topkindecies       = predictions.argsort()[topk*-1:][::-1]
        topkpredictions    = predictions[topkindecies]
        
        predictedLabels    = []
        for index in topkindecies:
            predictedLabels.append(classNames[index])
            
        print("Video name: ", "_".join(randomSequencePath.stem.split("_")[:-1]))
        print("Actual class: ", actualClass)
        print(f'Predicted class: {classNames[topkindecies[0]]} - {predictions[topkindecies[0]] *100}%')
        plotPredictions(topkpredictions, predictedLabels)
        print()
        
predictRNN()